# Word embedding task

Word embedding이란 단어의 단어의 뜻을 벡터로 표현하는 방법입니다. 
<img src="files/wv0.PNG">

단어 embedding 방법은 여러 가지가 있지만 본 task에서는 가장 기본적인 Word2Vec을 사용하여 학습을 진행하겠습니다. Word2Vec은 아래의 그림과 같이 주변 단어의 context를 고려하여 현재 위치의 단어를 예측하는 방식으로 학습합니다.

<img src="files/wv1.PNG">

Word2Vec 모델 구현에는 `gensim` 라이브러리를 사용합니다.

In [2]:
from gensim.models import Word2Vec

### Load corpus
학습 코퍼스는 위키피디아 문서를 형태소 분석을 하여 사용합니다.

In [2]:
corpus_file_name = "w2v/wiki_okt.txt"
corpus = []
with open(corpus_file_name, encoding="UTF8") as f:
    lines = f.readlines()
    print(len(lines))
    for line in lines:
        corpus.append(line.strip().split())

corpus[0:3]

287875


[['지미', '카터'],
 ['제임스',
  '얼',
  '"',
  '지미',
  '"',
  '카터',
  '주니어',
  '(,',
  '1924년',
  '10월',
  '1일',
  '~',
  ')',
  '는',
  '민주당',
  '출신',
  '미국',
  '39',
  '번째',
  '대통령',
  '(',
  '1977년',
  '~',
  '1981년',
  ')',
  '이다',
  '.'],
 ['지미',
  '카터',
  '는',
  '조지아주',
  '섬터',
  '카운티',
  '플레인스',
  '마을',
  '에서',
  '태어났다',
  '.',
  '조지아',
  '공과',
  '대학교',
  '를',
  '졸업',
  '하였다',
  '.',
  '그',
  '후',
  '해군',
  '에',
  '들어가',
  '전함',
  '·',
  '원자력',
  '·',
  '잠수함',
  '의',
  '승무원',
  '으로',
  '일',
  '하였다',
  '.',
  '1953년',
  '미국',
  '해군',
  '대위',
  '로',
  '예',
  '편하였고',
  '이후',
  '땅콩',
  '·',
  '면화',
  '등',
  '을',
  '가꿔',
  '많은',
  '돈',
  '을',
  '벌었다',
  '.',
  '그',
  '의',
  '별명',
  '이',
  '"',
  '땅콩',
  '농부',
  '"',
  '(',
  'Peanut',
  'Farmer',
  ')',
  '로',
  '알려졌다',
  '.']]

### Training
모델을 학습시키는 것은 간단합니다. 위에서 준비한 문서 list를 Word2Vec 모델에 바로 입력하면 됩니다.

그 외 hyperparameter를 간단히 설명하자면
* size: 단어 embedding 크기. 100으로 설정하면 한 단어에 100 dimension을 가지는 벡터를 얻을 수 있습니다.
* window: 단어 학습 시 고려할 주변 context 단어 개수
* min_count: 학습 corpus에 최소 min_count만큼 등장해야 최종 vocabulary list에 등록합니다.
* workers: 학습 processor 개수

In [3]:
model = Word2Vec(corpus, size=100, window=5, min_count=1, workers=4)
model.save("w2v.model")

### Testing
#### Representation
모델 학습이 끝나면 어떻게 단어가 표현되는지를 알아봅니다.

먼저, 단어 벡터를 불러오는 방식은 `model.wv[단어]`로 불러올 수 있습니다. 예를 들어, "하이닉스"라는 단어의 벡터를 알고 싶다면 아래와 같이 코드를 짜면 됩니다.

In [4]:
model.wv["하이닉스"]

array([-0.05458402,  0.16462323, -0.08330727,  0.01520697, -0.01744578,
        0.06041553,  0.09409352, -0.0339956 , -0.03857254,  0.10240825,
       -0.01074888, -0.07581771, -0.096007  , -0.0561431 ,  0.06756113,
        0.06963943, -0.12206154,  0.01840958,  0.0064967 , -0.03991486,
       -0.18301414, -0.05180284, -0.00646087, -0.13990559, -0.07753476,
       -0.06313062, -0.03087379,  0.06503866,  0.02186177,  0.2039582 ,
       -0.12392884, -0.10446624, -0.13538587,  0.01438549, -0.0498361 ,
        0.01280456,  0.08030511,  0.21595997,  0.05607784, -0.02687987,
        0.1178019 ,  0.01741406,  0.04175974,  0.09708944,  0.07883959,
        0.03563969,  0.00568025,  0.12903568,  0.07814391, -0.04740427,
        0.0579128 , -0.01110623, -0.0206186 , -0.01419968, -0.16259645,
       -0.00653264,  0.04595016,  0.0495457 , -0.00100294,  0.09331138,
        0.1140324 , -0.02521081, -0.01926421, -0.07037614, -0.11623224,
        0.00197264, -0.10430788,  0.18616222, -0.09523881, -0.08

위의 결과가 "하이닉스" 에 대한 벡터입니다. 하지만 이것만 보고는 단어의 뜻을 알기도 어렵고, 컴퓨터가 단어의 뜻을 제대로 학습했는지 알 수 없습니다. 따라서, 이 과정이 실제로 의미가 있는지 아래의 task를 통해 알아보겠습니다.

#### Similarity
Gensim의 word2vec 라이브러리는 다양한 similarity function을 지원합니다. 먼저, "하이닉스" 와 비슷한 단어를 찾아보겠습니다.

In [5]:
model.wv.similar_by_word("하이닉스")

[('텔레콤', 0.7518603801727295),
 ('브로드밴드', 0.7417868971824646),
 ('칼텍스', 0.7385914921760559),
 ('유플러스', 0.737071692943573),
 ('GS', 0.7230042815208435),
 ('SDS', 0.7191774845123291),
 ('케미칼', 0.6841263771057129),
 ('닛산', 0.6726300120353699),
 ('아반떼', 0.6720921993255615),
 ('BMW', 0.6636067628860474)]

SK의 다른 회사들이 상위권에 있는 것을 확인할 수 있습니다.

다음으로, 단어의 벡터를 더하고 빼는 방식으로 단어의 유사도를 측정할 수 있습니다. 예를 들어, 야구팀 "롯데 자이언츠" 에서 "롯데" 를 빼고 "SK" 를 넣으면 "와이번스" 가 나오면 좋을 것입니다.

In [6]:
model.wv.most_similar(positive=["자이언츠", "SK"], negative=["롯데"])

[('와이번스', 0.8660757541656494),
 ('타이거즈', 0.8638233542442322),
 ('KIA', 0.8458287715911865),
 ('해태', 0.8379089832305908),
 ('히어로즈', 0.8166918754577637),
 ('트윈스', 0.8111632466316223),
 ('넥센', 0.8111244440078735),
 ('이글스', 0.801300585269928),
 ('LG', 0.7984895706176758),
 ('베어스', 0.7875518798828125)]

마지막으로, SK와 하이닉스가 얼마나 유사한지 보겠습니다.

In [7]:
model.wv.similarity("SK", "하이닉스")

0.6111681

이제 학습된 단어 벡터를 가지고 다른 task를 수행해 보겠습니다.